# 케라스로 AlexNet 만들기

이 노트북에서 [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks)과 비슷한 심층 합성곱 신경망으로 [Oxford Flowers](http://www.robots.ox.ac.uk/~vgg/data/flowers/17/) 데이터셋의 꽃을 17개의 카테고리로 분류하겠습니다.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/dl-illustrated/blob/master/notebooks/10-2.alexnet_in_keras.ipynb)

#### 라이브러리를 적재합니다.

In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

#### 데이터를 적재하고 전처리합니다.

원서 노트북은 tflearn을 사용해 oxflower17 데이터셋을 다운로드합니다. 이 라이브러리는 텐서플로 2와 호환되지 않습니다. 여기에서는 사전에 tflearn으로 다운받은 데이터를 다운로드하여 사용합니다. 이 파일은 캐글 데이터셋에 업로드되어 있습니다(https://www.kaggle.com/haesunpark/oxflower17).

이 데이터셋에 대한 자세한 내용은 http://www.robots.ox.ac.uk/~vgg/data/flowers/17/ 을 참고하세요.

In [2]:
!rm oxflower17*
!wget https://bit.ly/31iVxQ1 -O oxflower17.npz.zip
!unzip oxflower17.npz.zip

--2020-08-21 09:54:41--  https://bit.ly/31iVxQ1
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://storage.googleapis.com/kaggle-data-sets/839629%2F1433323%2Fcompressed%2Foxflower17.npz.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1598260995&Signature=iKSJhi7W%2B%2BOTejsfj4cmv3kQtbAouPdGAOH33eJn3b2kRE28jwsSq%2BrOQbVAK3QYLE9uPn0ssO5zxHVKbvUiiodLCtqoQEpLxs4cbpEZubdWGZpLgb8DSMHf3hQF6rA5mn/T4xL4SLtPNYgRdWlCf5RQ%2BD3BtzlAApHqSiChabZk16%2BGJ3M6jij0RNhBLUg7cr5TDVAqlL4c8QP/3vfFQ7uwufUkL6fy28Ar8ZDA67N5HFp6UoAcPRbj1wyBMgF8zXMf9aRATCTyHhTHXjGokBJRo9wEP8eNCMulpZUIVGLuqNM0WC5iAyh1w/JXt/vMca90Q7IMQYkAac3dSSjrvg%3D%3D [following]
--2020-08-21 09:54:41--  https://storage.googleapis.com/kaggle-data-sets/839629%2F1433323%2Fcompressed%2Foxflower17.npz.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&

In [3]:
import numpy as np

data = np.load('oxflower17.npz')
X = data['X']
Y = data['Y']

In [4]:
X.shape, Y.shape

((1360, 224, 224, 3), (1360, 17))

#### 신경망 모델을 만듭니다.

In [5]:
model = Sequential()

model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))

model.add(Dense(17, activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 96)        384       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 256)       614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 10, 256)       1024      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 256)         5

#### 모델을 설정합니다.

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 훈련!

In [8]:
model.fit(X, Y, batch_size=64, epochs=100, verbose=1, validation_split=0.1, shuffle=True)

Epoch 1/100
20/20 [==============================] - 2s 93ms/step - loss: 4.5899 - accuracy: 0.2165 - val_loss: 8.5754 - val_accuracy: 0.0735
Epoch 2/100
20/20 [==============================] - 1s 37ms/step - loss: 3.3045 - accuracy: 0.2810 - val_loss: 4.4846 - val_accuracy: 0.1324
Epoch 3/100
20/20 [==============================] - 1s 39ms/step - loss: 2.6105 - accuracy: 0.3227 - val_loss: 4.2123 - val_accuracy: 0.1250
Epoch 4/100
20/20 [==============================] - 1s 38ms/step - loss: 2.4477 - accuracy: 0.3742 - val_loss: 3.3323 - val_accuracy: 0.1765
Epoch 5/100
20/20 [==============================] - 1s 38ms/step - loss: 1.9643 - accuracy: 0.4502 - val_loss: 2.4216 - val_accuracy: 0.4044
Epoch 6/100
20/20 [==============================] - 1s 35ms/step - loss: 1.9945 - accuracy: 0.4649 - val_loss: 2.9967 - val_accuracy: 0.2868
Epoch 7/100
20/20 [==============================] - 1s 34ms/step - loss: 1.9390 - accuracy: 0.4436 - val_loss: 2.7178 - val_accuracy: 0.3456
Epoch 

20/20 [==============================] - 1s 37ms/step - loss: 0.4224 - accuracy: 0.8840 - val_loss: 3.4882 - val_accuracy: 0.5368
Epoch 59/100
20/20 [==============================] - 1s 36ms/step - loss: 0.5003 - accuracy: 0.8685 - val_loss: 3.5730 - val_accuracy: 0.5147
Epoch 60/100
20/20 [==============================] - 1s 42ms/step - loss: 0.7077 - accuracy: 0.8374 - val_loss: 3.0363 - val_accuracy: 0.5809
Epoch 61/100
20/20 [==============================] - 1s 38ms/step - loss: 0.4870 - accuracy: 0.8660 - val_loss: 4.3815 - val_accuracy: 0.5221
Epoch 62/100
20/20 [==============================] - 1s 37ms/step - loss: 0.6476 - accuracy: 0.8464 - val_loss: 3.8621 - val_accuracy: 0.5000
Epoch 63/100
20/20 [==============================] - 1s 32ms/step - loss: 0.3292 - accuracy: 0.9036 - val_loss: 3.2083 - val_accuracy: 0.6176
Epoch 64/100
20/20 [==============================] - 1s 37ms/step - loss: 0.2506 - accuracy: 0.9281 - val_loss: 3.7084 - val_accuracy: 0.5441
Epoch 65/100